In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [17]:
spark = SparkSession.builder \
    .appName('Streaming data for word counting') \
    .master('spark://spark-master:7077') \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config('spark.sql.warehouse.dir', f's3a://lakehouse/') \
    .enableHiveSupport() \
    .getOrCreate()

In [7]:
spark

In [22]:
spark.stop()

In [18]:
lines = spark.readStream.format("socket")\
            .option("host", "192.168.17.1")\
            .option("port", "9999")\
            .load()

In [19]:
words_df = lines.select(split(col("value"), "\\s").alias("word"))
counts_df = words_df.groupBy("word").agg(count("word").alias("count"))

In [21]:
checkpoint_path = "s3a://lakehouse/streaming/test/checkpoint"

def process_batch(df, epoch_id):
    if not df.isEmpty(): # check if the dataframe is empty to avoid errors
      df.show()

streamingQuery = counts_df.writeStream\
                        .foreachBatch(process_batch)\
                        .format("console")\
                        .outputMode("complete")\
                        .trigger(processingTime="1 second")\
                        .option("checkpointlocation", checkpoint_path)\
                        .start()

# streamingQuery.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 